# Clustering coefficient

In [2]:
from queries import*
from utilities import*
from scipy.spatial import Delaunay
from numpy import linalg
from sklearn import preprocessing
import copy

# Extracting Clustering coefficients statistics per phenotype:
Extracting the mean, median and variance per phenotype

In [3]:
Directory = os.getcwd()
Directory_path = Directory + '/data/output'
files = os.listdir(Directory_path)


clustering_coef_data = pd.DataFrame({})

for file in files[:]:
    
    array = file.split('_')
    if len(array)==2:
        condition = array[0]
        PATIENT_ID = array[1]
    else:
        condition == ''
        
    
    if condition == 'DEEPMEL':
        
        
        print('Treating patient' + PATIENT_ID)
        OUTPUT_PATH = './data/output/DEEPMEL_' + PATIENT_ID +'/'
        print('Reading nodes')
        
        nodes = pd.read_csv(OUTPUT_PATH+'nodes_with_border.csv')
        print('DONE')           

        nodes.set_index('cell_id', inplace=True)

        print('Reading edges')
        edges = pd.read_csv(OUTPUT_PATH + 'graph_edges.csv')
        print('DONE')  

        edges = edges[edges.is_border== True]
        nodes = nodes[nodes.on_border == True]
    
        print('Building graph')
        cell_graph =nx.from_pandas_edgelist(edges, 'cell_id_1', 'cell_id_2', edge_attr=None, create_using= nx.Graph())
        print('DONE')

        print('Clustering coefficient extraction')
        clustering_coef = nx.clustering(cell_graph)


        clustering_df =  pd.DataFrame(clustering_coef.items(),columns=['cell_id','clustering_coef'])
        nodes = nodes.merge(clustering_df, on ='cell_id', how='left')

        clustering_df_grouped = nodes.groupby(['phenotype'])['clustering_coef'].agg(['mean','median','var']).reset_index()

        statistics = ['median','var']
        columns_to_keep = ['phenotype','mean']
        restricted = clustering_df_grouped[columns_to_keep]
        clustering_df_grouped_pivotted = pd.pivot_table(restricted, values='mean',columns=['phenotype']).reset_index()
        clustering_df_grouped_pivotted.columns.name = None
        clustering_df_grouped_pivotted = clustering_df_grouped_pivotted.drop(['index'],axis=1)
        columns = clustering_df_grouped_pivotted.columns
        new_columns = [column + '_mean' for column in columns]
        mapping ={}
        for old,new in zip(columns,new_columns):
            mapping[old] = new
        final = clustering_df_grouped_pivotted.rename(columns= mapping)    

        for stat in statistics:
            columns_to_keep = ['phenotype',stat]
            restricted = clustering_df_grouped[columns_to_keep]
            clustering_df_grouped_pivotted = pd.pivot_table(restricted, values=stat,columns=['phenotype']).reset_index()
            clustering_df_grouped_pivotted.columns.name = None
            clustering_df_grouped_pivotted = clustering_df_grouped_pivotted.drop(['index'],axis=1)
            columns = clustering_df_grouped_pivotted.columns
            new_columns = [column + '_'+ stat for column in columns]
            mapping ={}
            for old,new in zip(columns,new_columns):
                mapping[old] = new
    
            clustering_df_grouped_pivotted = clustering_df_grouped_pivotted.rename(columns= mapping)
            final = final.merge(clustering_df_grouped_pivotted,left_index=True, right_index=True)
            
        final['name'] = PATIENT_ID
        
        clustering_coef_data = pd.concat([clustering_coef_data,final])
        print('DONE')

        

Treating patient1C1
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1D1
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1E1
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1F1
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1J1
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1J2
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1K1
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1L1
Reading nodes
DONE
Reading edges
DONE
Building graph
DONE
Clustering coefficient extraction
DONE
Treating patient1M1
Reading nodes
DONE
Reading edges
DONE
Buildi

In [4]:
clustering_coef_data = clustering_coef_data.fillna(0)

In [5]:
clustering_coef_data.to_csv(Directory_path + '/clustering_coefficients_statistics.csv', index = False)

In [44]:
clustering_coef_data.columns

Index(['B_mean', 'DP1_mean', 'DP2_mean', 'MISSING_mean', 'T_mean',
       'macrophages_mean', 'stroma_mean', 'tumor_mean', 'B_median',
       'DP1_median', 'DP2_median', 'MISSING_median', 'T_median',
       'macrophages_median', 'stroma_median', 'tumor_median', 'B_var',
       'DP1_var', 'DP2_var', 'MISSING_var', 'T_var', 'macrophages_var',
       'stroma_var', 'tumor_var', 'name', 'dendtritic_mean',
       'dendtritic_median', 'dendtritic_var'],
      dtype='object')